In [ ]:
from pathlib import Path
import rasterio
import geopandas as gpd

data_dir = Path("../data")

# read DEM
dem = rasterio.open(data_dir / "oxf_dem.tif")

# have a look at the data?
dem.read()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

f, ax = plt.subplots()
flattened = dem.read().ravel()
ax.hist(flattened, bins=50)
ax.axvline(np.quantile(flattened, 0.95), ls='--', color='k', label="95th percentile")
ax.legend()
ax.set_xlabel("Elevation (m)")


In [ ]:
import osmnx

query = "Oxford, UK"
lat, lon = osmnx.geocoder.geocode(query)
tags = {"amenity": True}
points = osmnx.features.features_from_point((lat, lon), tags, 3000)
points.head()

In [ ]:
post_boxes = points.loc[points["amenity"] == "post_box", ["geometry"]].copy()
post_boxes.plot()

In [ ]:
import rasterio.plot

# plot our DEM and boundary data
fig, ax = plt.subplots()
rasterio.plot.show(dem, ax=ax)
post_boxes.to_crs(dem.crs).plot(ax=ax, color='r')

In [19]:
import rasterstats

# raster extraction (points)
result = rasterstats.point_query(
    post_boxes.to_crs(dem.crs),
    dem.read(1),
    affine = dem.transform,
)
post_boxes["height_m"] = result

In [ ]:
post_boxes

In [ ]:
# point height distribution
post_boxes.height_m.plot(kind="hist")

In [ ]:
import shapely

# extract a transect
coords = [[-1.33, 51.77], [-1.18, 51.75]]
transect = shapely.LineString(coords)
transect_gdf = gpd.GeoSeries(transect, crs=4326)
print(transect_gdf)

In [ ]:
# project to CRS in meters
transect_proj_gdf = transect_gdf.to_crs(transect_gdf.estimate_utm_crs())
transect_proj = transect_proj_gdf.iloc[0]
print(transect_proj_gdf)

In [ ]:
# get 250m intervals of distance along transect
distances = np.arange(0, transect_proj.length, 250)
distances[:5]

In [ ]:
transect_pnt = [transect_proj.interpolate(distance) for distance in distances]
transect_pnt = gpd.GeoSeries(transect_pnt, crs=transect_proj_gdf.crs).to_crs(dem.crs)
transect_pnt.head()

In [ ]:
import rasterstats

# sample the heights along the transect
result = rasterstats.point_query(
    transect_pnt,
    dem.read(1),
    nodata = dem.nodata,
    affine = dem.transform,
    interpolate='nearest'
)
# save the result to a GeoDataFrame
transect = gpd.GeoDataFrame(geometry=transect_pnt.geometry)
transect['dist'] = distances
transect['elev'] = result
transect.head()

In [ ]:
f, ax = plt.subplots()
ax.plot(transect.dist, transect.elev)
ax.set_xlabel("Distance [m]")
ax.set_ylabel("Elevation [m]")
ax.grid()

In [ ]:
f, ax = plt.subplots()
rasterio.plot.show(dem, ax=ax)
transect.plot(ax=ax, color='white', markersize=2)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")